In [1]:
import os
import numpy as np

In [2]:
from preprocessing import HOGFaceDetector, HaarFaceDetector, FaceAligner, convert_to_RGB

from PIL import Image
import cv2

In [3]:
hog_face_detector = HOGFaceDetector()
haar_face_detector = HaarFaceDetector('resources/haarcascade_frontalface_default.xml')
face_aligner = FaceAligner(dlib_predictor_path="resources/shape_predictor_68_face_landmarks.dat", face_template_path = "resources/face_template.npy")

In [4]:
IMAGES_SOURCE = './data/lfw_divided'

TRAIN_PATH = './data/lfw_divided/lfw_train/'
TEST_PATH = './data/lfw_divided/lfw_test/'

FACES_TRAIN_PATH = './data/lfw_cropped_and_aligned_faces/train/'
FACES_TEST_PATH = './data/lfw_cropped_and_aligned_faces/test/'

In [ ]:
def save_image_into_folder(root, ndarray_image, name_image_file, path_to_folder):
    image = Image.fromarray(ndarray_image, 'RGB')

    new_dir_path_to_image = os.path.join(path_to_folder, os.path.basename(root))
    
    if not os.path.exists(new_dir_path_to_image):
        os.makedirs(new_dir_path_to_image)
  
    image.save(new_dir_path_to_image + '/'+ name_image_file, 'JPEG')    
        
for root, dirs, files in os.walk(IMAGES_SOURCE):
    for directory in dirs:
        level = 0
        for directory_root, directory_dirs, directory_files in os.walk(root+ '/'+ directory):      
            if level != 0:
                for file in directory_files:
                    image = cv2.imread(directory_root + '/' + file)

                    face_rects = hog_face_detector.detect_faces(image, upscale_factor=2, get_top = 1)
                    if len(face_rects) == 0:
                        face_rects = haar_face_detector.detect_faces(image, get_top = 1)
                        if len(face_rects) == 0:
                            face_rects = haar_face_detector.detect_faces(image, scaleFactor = 1.3, minNeighbors = 3, get_top = 1)
                            if len(face_rects) == 0:
                                face_rects = haar_face_detector.detect_faces(image, scaleFactor = 1.1, minNeighbors = 2, get_top = 1)
                            if len(face_rects) == 0:
                                face_rects = haar_face_detector.detect_faces(image, scaleFactor = 1.1, minNeighbors = 1, get_top = 1)
                            
                    faces = face_aligner.align_faces(image, face_rects)
                    print(file)
                    faces[0] = convert_to_RGB(faces[0].astype(np.uint8))
                    if directory == 'lfw_train':
                        save_image_into_folder(directory_root, faces[0], file, FACES_TRAIN_PATH)
                    elif directory == 'lfw_test':
                        save_image_into_folder(directory_root, faces[0], file, FACES_TEST_PATH)
            level = 1  

    break
